# Who is NOT using the LMS?
This question came from the head of Faculty Development. It was just after Spring Break, with one extra week for Faculty to shift. 
* Which courses aren't being used?
* How many students are affected?
* Who is teaching these courses?
* What department are they in?


The query: 
```sql
SELECT SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, '-[A-Z&]+'),2) COURSE_UNIT,
       SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, ' [A-Z]{3}$'),2) COURSE_TYPE,
       REPLACE(cm.COURSE_NAME,',',' ') COURSE_NAME,
       cm.COURSE_ID,
       cm.AVAILABLE_IND,
       MAX(cu.LAST_ACCESS_DATE) last_access,
       SUM(students.students)/COUNT(students.students) student_count,
       listagg(u.email, ';') WITHIN GROUP (order by u.email) emails,
       listagg(u.firstname || ' ' || u.LASTNAME, ';') WITHIN GROUP (order by u.email) names

FROM BB_BB60.USERS u inner join BB_BB60.COURSE_USERS cu on u.pk1 = cu.USERS_PK1
inner join BB_BB60.COURSE_MAIN cm on cu.CRSMAIN_PK1 = cm.pk1
left join (select crsmain_pk1, count(pk1) students
    from bb_bb60.COURSE_USERS
    where role='S'
    and row_status = 0
    group by crsmain_pk1) students on students.crsmain_pk1 = cm.pk1

WHERE cm.COURSE_NAME LIKE '2020Spring-%' AND (cu.ROLE='P' OR cu.ROLE='T') AND
      NOT EXISTS (SELECT cc.crsmain_pk1 course_id
      from bb_bb60.COURSE_COURSE cc WHERE cc.CRSMAIN_PK1 = cm.pk1)

GROUP BY cm.COURSE_NAME, cm.COURSE_ID, cm.AVAILABLE_IND
HAVING SUM(students.students)/COUNT(students.students) IS NOT NULL
ORDER BY student_count desc
```

## Setup
Connect to the database, using env setup

In [2]:
from sqlalchemy import create_engine
from os import getenv as ge
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
bbuser, bbpass, bbhost = ge('bbuser'), ge('bbpass'), ge('bbhost')
con_string = f'oracle+cx_oracle://{bbuser}:{bbpass}@{bbhost}'
engine = create_engine(con_string, max_identifier_length=128)

### Which courses aren't being used?

In [4]:
query = """
SELECT cm.COURSE_ID, MAX(cu.LAST_ACCESS_DATE) last_access
FROM BB_BB60.USERS u 
    inner join BB_BB60.COURSE_USERS cu on u.pk1 = cu.USERS_PK1
    inner join BB_BB60.COURSE_MAIN cm on cu.CRSMAIN_PK1 = cm.PK1
    WHERE cm.COURSE_NAME LIKE '2020Summer-%' AND (cu.ROLE='P' OR cu.ROLE='T')
GROUP BY cm.COURSE_ID
"""

pd.read_sql(query, con=engine).head()

,course_id,last_access
0,4206-72447,NaT
1,4206-76620,NaT
2,4206-70666,NaT
3,4206-70667,NaT
4,4206-70674,NaT


### Problems
* Course_ID doesn't have much information for sorting.
* Not sure who to contact for each course.

In [5]:
query = """
SELECT cm.COURSE_ID, 
       SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, '-[A-Z&]+'),2) COURSE_UNIT, -- based on course name format
       SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, ' [A-Z]{3}$'),2) COURSE_TYPE, -- based on course name format
       REPLACE(cm.COURSE_NAME,',',' ') COURSE_NAME,MAX(cu.LAST_ACCESS_DATE) last_access, -- Commas break csv output
       listagg(u.email, ';') WITHIN GROUP (order by u.email) emails,
       listagg(u.firstname || ' ' || u.LASTNAME, ';') WITHIN GROUP (order by u.email) names -- Listagg inlcuded multiple results in one row.

FROM BB_BB60.USERS u 
    inner join BB_BB60.COURSE_USERS cu on u.pk1 = cu.USERS_PK1
    inner join BB_BB60.COURSE_MAIN cm on cu.CRSMAIN_PK1 = cm.PK1
    WHERE cm.COURSE_NAME LIKE '2020Summer-%' AND (cu.ROLE='P' OR cu.ROLE='T')

GROUP BY cm.COURSE_ID, cm.COURSE_NAME
"""

pd.read_sql(query, con=engine).head()

,course_id,course_unit,course_type,course_name,last_access,emails,names
0,4206-70002,AAAS,IND,2020Summer-AAAS 460 Topics and Problems in Afr...,NaT,ukpokodu@ku.edu,Peter Ukpokodu
1,4206-70003,AAAS,IND,2020Summer-AAAS 690 Investigation and Conferen...,NaT,ukpokodu@ku.edu,Peter Ukpokodu
2,4206-70004,AE,LEC,2020Summer-AE 345 Fluid Mechanics LEC,2020-07-10 11:13:38,l215z945@ku.edu;rtaghavi@ku.edu,Lu Zhao;Ray Taghavi
3,4206-70005,AE,IND,2020Summer-AE 592 Special Projects in Aerospac...,NaT,sfarokhi@ku.edu,Saeed Farokhi
4,4206-70006,AE,IND,2020Summer-AE 592 Special Projects in Aerospac...,NaT,rhale@ku.edu,Richard Hale


### Problems
* Includes courses with 0 student enrollments.
* Includes child courses (KU has many merged courses)

In [6]:
query = """
SELECT SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, '-[A-Z&]+'),2) COURSE_UNIT,
       SUBSTR(REGEXP_SUBSTR(cm.COURSE_NAME, ' [A-Z]{3}$'),2) COURSE_TYPE,
       REPLACE(cm.COURSE_NAME,',',' ') COURSE_NAME,
       cm.COURSE_ID,
       cm.AVAILABLE_IND,
       MAX(cu.LAST_ACCESS_DATE) last_access,
       SUM(students.students)/COUNT(students.students) student_count, --Collapses per course, due to the access dates 
       listagg(u.email, ';') WITHIN GROUP (order by u.email) emails,
       listagg(u.firstname || ' ' || u.LASTNAME, ';') WITHIN GROUP (order by u.email) names

FROM BB_BB60.USERS u inner join BB_BB60.COURSE_USERS cu on u.pk1 = cu.USERS_PK1
inner join BB_BB60.COURSE_MAIN cm on cu.CRSMAIN_PK1 = cm.pk1

left join (select crsmain_pk1, count(pk1) students
    from bb_bb60.COURSE_USERS
    where role='S'
    and row_status = 0
    group by crsmain_pk1) students on students.crsmain_pk1 = cm.pk1 -- will return number of students per course | subquery

WHERE cm.COURSE_NAME LIKE '2020Summer-%' AND (cu.ROLE='P' OR cu.ROLE='T') AND
      NOT EXISTS (SELECT cc.crsmain_pk1 course_id
      from bb_bb60.COURSE_COURSE cc WHERE cc.CRSMAIN_PK1 = cm.pk1) -- Removes child courses.

GROUP BY cm.COURSE_NAME, cm.COURSE_ID, cm.AVAILABLE_IND
HAVING SUM(students.students)/COUNT(students.students) IS NOT NULL -- ensures there are students in the course
ORDER BY student_count desc --shows student count
"""

pd.read_sql(query, con=engine).head()


,course_unit,course_type,course_name,course_id,available_ind,last_access,student_count,emails,names
0,PHSX,LBN,2020Summer-PHSX 114 College Physics I LBN,4206-71277,Y,2020-07-13 14:26:19,116,ColeLindsey@ku.edu;am.morgan@ku.edu;b047m507@k...,Cole Lindsey;Aaron Morgan;Brendon Madison;Carl...
1,PHSX,LEC,2020Summer-PHSX 114 College Physics I LEC,4206-71276,N,2020-07-13 12:50:19,106,a665l723@ku.edu;shark@ku.edu;slegres@ku.edu;su...,Ann Lindbloom;Christopher Fischer;Sarah LeGres...
2,PHAR,FLD,2020Summer-PHAR 550 Introductory Pharmacy Prac...,4206-72717,N,NaT,104,jheidric@ku.edu,Joseph Heidrick
3,ECON,LEC,2020Summer-ECON 142 Principles of Microeconomi...,4206-70484,Y,2020-07-07 18:58:27,103,j_lugovskyy@ku.edu;joseoyeon@ku.edu,Josephine Lugovskyy;SeoYeon Jo
4,PHSX,LEC,2020Summer-PHSX 212 General Physics II LEC,4206-74307,Y,2020-07-13 12:13:42,101,a665l723@ku.edu;j743d550@ku.edu;sandhyaravikum...,Ann Lindbloom;Jennifer Delgado;Sandhya Ravikum...


### Not demonstrated:
Pull info from Campus Solutions to get Academic Org. Break results down to send to each Dean.